In [1]:
import sqlite3
import sqlalchemy
import pandas as pd
from seaborn import load_dataset
from IPython.display import display
pd.set_option('max_row', 5)

## データ取得で主に使用するライブラリ
---
Python でデータ分析を行う場合には主に SQL と pandas を使用する。

<table class="border text-center">
    <tr>
        <th class="border-top-none border-left-none border-right-bold border-bottom-bold background-default"></th>
        <th>メリット</th>
        <th>デメリット</th>
    </tr>
    <tr>
        <th class="border-right-bold border-bottom">表計算ソフト</th>
        <td>習得が簡単</td>
        <td>小さいデータしか扱えない</td>
    </tr>
    <tr>
        <th class="border-right-bold border-bottom">pandas</th>
        <td>Python だけで完結する</td>
        <td><strong>メモリに収まるデータしか扱えない</strong></td>
    </tr>
    <tr>
        <th class="border-right-bold border-bottom">SQL</th>
        <td><strong>大規模データも扱える</strong></td>
        <td>習得するものが増える</td>
    </tr>
</table>

SQL を使ったデータベースには複数の種類があるが、以下では Python の標準モジュールに採用されている SQLite3 を使用する。他のデータベースでも基本的な部分は同じ。  
Python の代表的な ORM である SQLAlchemy を使うと、 Python でプログラムを書くようにデータベースを扱うこともできる。また、 pandas からデータベースに接続する場合には SQLAlchemy のエンジンが必要。

## データベースへの接続
---
`sqlite3.connect`を使用する。

In [2]:
sqlite3.connect??

In [3]:
con = sqlite3.connect('data/iris.sqlite3')

SQL 文を実行するためには`cursor`を取得する。

In [4]:
cur = con.cursor()
cur.execute('SELECT * FROM iris LIMIT 5').fetchall()

[(5.1, 3.5, 1.4, 0.2, 'setosa'),
 (4.9, 3.0, 1.4, 0.2, 'setosa'),
 (4.7, 3.2, 1.3, 0.2, 'setosa'),
 (4.6, 3.1, 1.5, 0.2, 'setosa'),
 (5.0, 3.6, 1.4, 0.2, 'setosa')]

使用し終えたら必ず接続を閉じる。閉じ忘れるとファイルがロックされたままになる。

In [5]:
con.close()

SQLite 以外で実行するときのため、 SQLAlchemy を使って同様の操作を行う例を示す。

In [6]:
engine = sqlalchemy.create_engine('sqlite:///data/iris.sqlite3')
con = engine.raw_connection()
cur = con.cursor()
cur.execute('SELECT * FROM iris LIMIT 5').fetchall()

[(5.1, 3.5, 1.4, 0.2, 'setosa'),
 (4.9, 3.0, 1.4, 0.2, 'setosa'),
 (4.7, 3.2, 1.3, 0.2, 'setosa'),
 (4.6, 3.1, 1.5, 0.2, 'setosa'),
 (5.0, 3.6, 1.4, 0.2, 'setosa')]

In [7]:
con.close()

ただし、 SQL文を実行するだけなら`engine.execute`で可能。

In [8]:
engine.execute('SELECT * FROM iris LIMIT 5').fetchall()

[(5.1, 3.5, 1.4, 0.2, 'setosa'),
 (4.9, 3.0, 1.4, 0.2, 'setosa'),
 (4.7, 3.2, 1.3, 0.2, 'setosa'),
 (4.6, 3.1, 1.5, 0.2, 'setosa'),
 (5.0, 3.6, 1.4, 0.2, 'setosa')]

## データ読み込み

### pandas
---
`pandas.read_csv`( CSV ファイル ) や`pandas.read_sql_table`( データベース ) を使用する。  
CSV や SQL 以外にも様々な形式を読み込める。

In [9]:
[att for att in dir(pd) if att.startswith('read_')]

['read_clipboard',
 'read_csv',
 'read_excel',
 'read_feather',
 'read_fwf',
 'read_gbq',
 'read_hdf',
 'read_html',
 'read_json',
 'read_msgpack',
 'read_parquet',
 'read_pickle',
 'read_sas',
 'read_sql',
 'read_sql_query',
 'read_sql_table',
 'read_stata',
 'read_table']

In [10]:
pd.read_csv??

In [11]:
pd.read_csv('data/Titanic.csv')

,Class,Sex,Age,Survived
1,3rd,Male,Child,No
2,3rd,Male,Child,No
...,...,...,...,...
2200,Crew,Female,Adult,Yes
2201,Crew,Female,Adult,Yes


#### CSVファイル読み込みでよく利用するオプション
---
詳細は[ドキュメント](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)を参照。

<table class="text-left background-default border">
    <tr>
        <td><code>header</code></td>
        <td>列名をどうするか<br />デフォルトは自動で推論<br />列名として使用する行の番号を与える</td>
    </tr>
    <tr>
        <td><code>names</code></td>
        <td>列名の指定<br />列名の入ったリストを与える</td>
    </tr>
    <tr>
        <td><code>index_col</code></td>
        <td>行名として使用する列<br />列の番号を与える</td>
    </tr>
    <tr>
        <td><code>usecols</code></td>
        <td>読み込む列の指定<br />列番号や列名のリストを与える</td>
    </tr>
    <tr>
        <td><code>skipinitialspace</code></td>
        <td>コンマなどの区切り文字の後のスペースを削除するか<br />真偽値を与える</td>
    </tr>
    <tr>
        <td><code>skiprows</code></td>
        <td>スキップする行の指定<br />スキップする行番号のリストや先頭からスキップする行数を与える</td>
    </tr>
    <tr>
        <td><code>na_values</code></td>
        <td>欠損値の指定<br />読み込み時に欠損値として扱いたい文字列などを与える</td>
    </tr>
    <tr>
        <td><code>parse_dates</code></td>
        <td>日付型データへの変換の指定<br />行名を変換する場合は真偽値を与える<br />特定の列を変換する場合は行番号や行名のリストなどを与える</td>
    </tr>
    <tr>
        <td><code>iterator</code></td>
        <td>ファイルサイズが大きい場合などにイテレータとして一部ずつ読み込むかどうか<br />真偽値を与える</td>
    </tr>
    <tr>
        <td><code>chunksize</code></td>
        <td>イテレータで一度に読み込む行数の指定<br />数値を与える</td>
    </tr>
    <tr>
        <td><code>comment</code></td>
        <td>コメントとしてスキップする行の指定<br /># などコメント行を表す文字列を与える</td>
    </tr>
    <tr>
        <td><code>encoding</code></td>
        <td>文字列エンコードの指定</td>
    </tr>
</table>

### SQL
---
`pandas.read_sql_table`か`pandas.read_sql`から`SELECT`文を使用する。  
`SELECT`に続けて列名を列挙し、その後`FROM`に続けてテーブル名 (表計算ソフトのシートに相当) を指定する。列名に * (アスタリスク) を使うと全ての列を指定できる。

In [12]:
pd.read_sql_table??

In [13]:
engine = sqlalchemy.create_engine('sqlite:///data/iris.sqlite3')

In [14]:
pd.read_sql_table('iris', engine)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
...,...,...,...,...,...
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


In [15]:
pd.read_sql??

In [16]:
pd.read_sql('SELECT * FROM iris', engine)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
...,...,...,...,...,...
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


## 特定のカラムを取得

### pandas
---

In [17]:
iris = load_dataset('iris')
print('iris')
display(iris)

iris


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
...,...,...,...,...,...
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


1 列のみ取得

In [18]:
iris['species']

0         setosa
1         setosa
         ...    
148    virginica
149    virginica
Name: species, Length: 150, dtype: object

In [19]:
iris.loc[:, 'species']

0         setosa
1         setosa
         ...    
148    virginica
149    virginica
Name: species, Length: 150, dtype: object

In [20]:
iris.iloc[:, -1]

0         setosa
1         setosa
         ...    
148    virginica
149    virginica
Name: species, Length: 150, dtype: object

複数列取得

In [21]:
iris[['sepal_length', 'sepal_width']]

,sepal_length,sepal_width
0,5.1,3.5
1,4.9,3.0
...,...,...
148,6.2,3.4
149,5.9,3.0


In [22]:
iris.loc[:, ['sepal_length', 'sepal_width']]

,sepal_length,sepal_width
0,5.1,3.5
1,4.9,3.0
...,...,...
148,6.2,3.4
149,5.9,3.0


In [23]:
iris.loc[:, 'sepal_length':'sepal_width']

,sepal_length,sepal_width
0,5.1,3.5
1,4.9,3.0
...,...,...
148,6.2,3.4
149,5.9,3.0


In [24]:
iris.iloc[:, [0, 1]]

,sepal_length,sepal_width
0,5.1,3.5
1,4.9,3.0
...,...,...
148,6.2,3.4
149,5.9,3.0


In [25]:
iris.iloc[:, :2]

,sepal_length,sepal_width
0,5.1,3.5
1,4.9,3.0
...,...,...
148,6.2,3.4
149,5.9,3.0


### SQL
---

In [26]:
engine = sqlalchemy.create_engine('sqlite:///data/iris.sqlite3')

1 列のみ取得

In [27]:
pd.read_sql_table('iris', engine, columns=['species'])

,species
0,setosa
1,setosa
...,...
148,virginica
149,virginica


In [28]:
pd.read_sql('SELECT species FROM iris', engine)

,species
0,setosa
1,setosa
...,...
148,virginica
149,virginica


複数列取得

In [29]:
pd.read_sql_table('iris', engine, columns=['sepal_length', 'sepal_width'])

,sepal_length,sepal_width
0,5.1,3.5
1,4.9,3.0
...,...,...
148,6.2,3.4
149,5.9,3.0


In [30]:
pd.read_sql('SELECT sepal_length, sepal_width FROM iris', engine)

,sepal_length,sepal_width
0,5.1,3.5
1,4.9,3.0
...,...,...
148,6.2,3.4
149,5.9,3.0


## 不要なカラムの削除
---
データ分析者がデータベースに変更を加えることはほとんどないため、ここでは pandas で一旦データを読み込んだ後に不要なものを取り除く方法のみ扱う。

`pandas.DataFrame.drop`を使用するが、 inplace 引数を True にするとそのデータフレームの値を削除し、 False にすると列 (行) を削除したコピーを返す。 True の場合、メモリを節約できるが、途中でやり直したい場合にはまたデータ読み込みから始めなければならない。 False の場合はその逆。

In [31]:
pd.DataFrame.drop??

In [32]:
iris.drop('species', axis=1)

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
...,...,...,...,...
148,6.2,3.4,5.4,2.3
149,5.9,3.0,5.1,1.8


In [33]:
iris.drop(columns='species')

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
...,...,...,...,...
148,6.2,3.4,5.4,2.3
149,5.9,3.0,5.1,1.8


In [34]:
iris_copy = iris.copy()
iris_copy.drop('species', axis=1, inplace=True)
iris_copy

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
...,...,...,...,...
148,6.2,3.4,5.4,2.3
149,5.9,3.0,5.1,1.8


## データ保存
---
`pandas.DataFrame.to_csv`や`pandas.DataFrame.to_sql`などを使用する。  
ただし、分析者が保存元のデータを変更することはまずないので、 SQL で保存する場合でも別の手元のデータベースを使う。

In [35]:
pd.DataFrame.to_csv??

In [36]:
iris.to_csv('data/iris_copy.csv')

In [37]:
pd.DataFrame.to_sql??

In [38]:
engine = sqlalchemy.create_engine('sqlite:///data/iris_copy.sqlite3')
iris.to_sql('iris_copy', engine, if_exists='replace')

###### 練習問題

[アメリカの統計情報](https://factfinder.census.gov/faces/nav/jsf/pages/guided_search.xhtml)から以下の条件で人口統計のデータを探し、「SEX BY AGE BY VETERAN STATUS FOR THE CIVILIAN POPULATION 18 YEARS AND OVER」の最新のものをダウンロードする。
- People:Basic Count/Estimate: Civilian Population
- All available races
- All States within United States, Puerto Rico, and the Island Areas

ダウンロードしたCSVファイルからデータを 100 件ずつ読み込み、随時`data/census.db`というデータベースに保存する。データベースへの保存には`pandas.DataFrame.to_sql`を使用する。  
(サイズの大きいCSVファイルや複数に分かれているデータをまとめる練習)

In [39]:
pd.DataFrame.to_sql??

In [40]:
reader = pd.read_csv('data/ACS_15_SPT_B21001_with_ann.csv', skiprows=1, chunksize=100)
engine = sqlalchemy.create_engine('sqlite:///data/census-example.db')
for i, chunk in enumerate(reader):
    chunk.to_sql('census', engine, if_exists='replace' if i is 0 else 'append', index=False)

## データ絞り込み

### 件数指定

In [41]:
titanic = pd.read_csv('data/Titanic.csv')
print('titanic')
display(titanic)

titanic


,Class,Sex,Age,Survived
1,3rd,Male,Child,No
2,3rd,Male,Child,No
...,...,...,...,...
2200,Crew,Female,Adult,Yes
2201,Crew,Female,Adult,Yes


#### pandas
---
`head`や`iloc`を使用する。

In [42]:
titanic.head(3)

,Class,Sex,Age,Survived
1,3rd,Male,Child,No
2,3rd,Male,Child,No
3,3rd,Male,Child,No


In [43]:
titanic.iloc[:3]

,Class,Sex,Age,Survived
1,3rd,Male,Child,No
2,3rd,Male,Child,No
3,3rd,Male,Child,No


#### SQL
---
`LIMIT`句を使用する。

In [44]:
engine = sqlalchemy.create_engine('sqlite:///data/iris.sqlite3')
pd.read_sql('SELECT * FROM iris LIMIT 3', engine)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa


### ソート

In [45]:
mpg = load_dataset('mpg')
print('mpg')
display(mpg)

mpg


,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
...,...,...,...,...,...,...,...,...,...
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger
397,31.0,4,119.0,82.0,2720,19.4,82,usa,chevy s-10


#### pandas
---
`pd.DataFrame.sort_values`を使用する。

In [46]:
pd.DataFrame.sort_values??

昇順

In [47]:
mpg.sort_values('acceleration')

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
11,14.0,8,340.0,160.0,3609,8.0,70,usa,plymouth 'cuda 340
9,15.0,8,390.0,190.0,3850,8.5,70,usa,amc ambassador dpl
...,...,...,...,...,...,...,...,...,...
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
299,27.2,4,141.0,71.0,3190,24.8,79,europe,peugeot 504


降順

In [48]:
mpg.sort_values('acceleration', ascending=False)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
299,27.2,4,141.0,71.0,3190,24.8,79,europe,peugeot 504
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
...,...,...,...,...,...,...,...,...,...
9,15.0,8,390.0,190.0,3850,8.5,70,usa,amc ambassador dpl
11,14.0,8,340.0,160.0,3609,8.0,70,usa,plymouth 'cuda 340


上位 5 件の取得は以下のように行う。

In [49]:
mpg.sort_values('acceleration', ascending=False).head(5)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
299,27.2,4,141.0,71.0,3190,24.8,79,europe,peugeot 504
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
326,43.4,4,90.0,48.0,2335,23.7,80,europe,vw dasher (diesel)
59,23.0,4,97.0,54.0,2254,23.5,72,europe,volkswagen type 3
300,23.9,8,260.0,90.0,3420,22.2,79,usa,oldsmobile cutlass salon brougham


#### SQL
---
`ORDER BY`を使用する。

In [50]:
mpg_db = sqlalchemy.create_engine('sqlite:///data/mpg.sqlite3')

昇順

In [51]:
pd.read_sql('SELECT * FROM mpg ORDER BY acceleration', mpg_db)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,14.0,8,340.0,160.0,3609,8.0,70,usa,plymouth 'cuda 340
1,14.0,8,440.0,215.0,4312,8.5,70,usa,plymouth fury iii
...,...,...,...,...,...,...,...,...,...
396,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
397,27.2,4,141.0,71.0,3190,24.8,79,europe,peugeot 504


In [52]:
pd.read_sql('SELECT * FROM mpg ORDER BY acceleration ASC', mpg_db)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,14.0,8,340.0,160.0,3609,8.0,70,usa,plymouth 'cuda 340
1,14.0,8,440.0,215.0,4312,8.5,70,usa,plymouth fury iii
...,...,...,...,...,...,...,...,...,...
396,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
397,27.2,4,141.0,71.0,3190,24.8,79,europe,peugeot 504


降順

In [53]:
pd.read_sql('SELECT * FROM mpg ORDER BY acceleration DESC', mpg_db)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,27.2,4,141.0,71.0,3190,24.8,79,europe,peugeot 504
1,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
...,...,...,...,...,...,...,...,...,...
396,15.0,8,390.0,190.0,3850,8.5,70,usa,amc ambassador dpl
397,14.0,8,340.0,160.0,3609,8.0,70,usa,plymouth 'cuda 340


上位 5 件の取得は以下のように行う。

In [54]:
pd.read_sql('SELECT * FROM mpg ORDER BY acceleration DESC LIMIT 5', mpg_db)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,27.2,4,141.0,71.0,3190,24.8,79,europe,peugeot 504
1,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
2,43.4,4,90.0,48.0,2335,23.7,80,europe,vw dasher (diesel)
3,23.0,4,97.0,54.0,2254,23.5,72,europe,volkswagen type 3
4,29.0,4,85.0,52.0,2035,22.2,76,usa,chevrolet chevette


### 条件指定

#### pandas
---
`loc`や`query`を使用する。

##### 特定の値のデータ
---
`loc`

In [55]:
titanic.loc[titanic['Class'] == '1st']

,Class,Sex,Age,Survived
53,1st,Male,Adult,No
54,1st,Male,Adult,No
...,...,...,...,...
2024,1st,Female,Adult,Yes
2025,1st,Female,Adult,Yes


条件文の評価結果は真偽値の配列。

In [56]:
titanic['Class'] == '1st'

1       False
2       False
        ...  
2200    False
2201    False
Name: Class, Length: 2201, dtype: bool

`query`

In [57]:
titanic.query('Class == "1st"')

,Class,Sex,Age,Survived
53,1st,Male,Adult,No
54,1st,Male,Adult,No
...,...,...,...,...
2024,1st,Female,Adult,Yes
2025,1st,Female,Adult,Yes


`@`をつけると Python の変数も使用可能。

In [58]:
c = '1st'
titanic.query('Class == @c')

,Class,Sex,Age,Survived
53,1st,Male,Adult,No
54,1st,Male,Adult,No
...,...,...,...,...
2024,1st,Female,Adult,Yes
2025,1st,Female,Adult,Yes


##### 特定の値以外のデータ

In [59]:
titanic.loc[titanic['Class'] != 'Crew', ['Class', 'Survived']]

,Class,Survived
1,3rd,No
2,3rd,No
...,...,...
2180,3rd,Yes
2181,3rd,Yes


In [60]:
titanic.query('Class != "Crew"')[['Class', 'Survived']]

,Class,Survived
1,3rd,No
2,3rd,No
...,...,...
2180,3rd,Yes
2181,3rd,Yes


##### リストに含まれる値を持つデータ

In [61]:
l = ['1st', '2nd']

In [62]:
titanic.loc[titanic['Class'].isin(l)]

,Class,Sex,Age,Survived
53,1st,Male,Adult,No
54,1st,Male,Adult,No
...,...,...,...,...
2104,2nd,Female,Adult,Yes
2105,2nd,Female,Adult,Yes


In [63]:
titanic.query('Class in @l')

,Class,Sex,Age,Survived
53,1st,Male,Adult,No
54,1st,Male,Adult,No
...,...,...,...,...
2104,2nd,Female,Adult,Yes
2105,2nd,Female,Adult,Yes


In [64]:
titanic.query('Class == @l')

,Class,Sex,Age,Survived
53,1st,Male,Adult,No
54,1st,Male,Adult,No
...,...,...,...,...
2104,2nd,Female,Adult,Yes
2105,2nd,Female,Adult,Yes


##### リストに含まれる値を除くデータ

In [65]:
titanic.loc[~titanic['Class'].isin(l)]

,Class,Sex,Age,Survived
1,3rd,Male,Child,No
2,3rd,Male,Child,No
...,...,...,...,...
2200,Crew,Female,Adult,Yes
2201,Crew,Female,Adult,Yes


In [66]:
titanic.query('Class not in @l')

,Class,Sex,Age,Survived
1,3rd,Male,Child,No
2,3rd,Male,Child,No
...,...,...,...,...
2200,Crew,Female,Adult,Yes
2201,Crew,Female,Adult,Yes


In [67]:
titanic.query('Class != @l')

,Class,Sex,Age,Survived
1,3rd,Male,Child,No
2,3rd,Male,Child,No
...,...,...,...,...
2200,Crew,Female,Adult,Yes
2201,Crew,Female,Adult,Yes


##### 一定の範囲内のデータ

In [68]:
iris.loc[(2 < iris['petal_length']) & (iris['petal_length'] < 4)]

,sepal_length,sepal_width,petal_length,petal_width,species
57,4.9,2.4,3.3,1.0,versicolor
59,5.2,2.7,3.9,1.4,versicolor
...,...,...,...,...,...
93,5.0,2.3,3.3,1.0,versicolor
98,5.1,2.5,3.0,1.1,versicolor


In [69]:
iris.query('2 < petal_length < 4')

,sepal_length,sepal_width,petal_length,petal_width,species
57,4.9,2.4,3.3,1.0,versicolor
59,5.2,2.7,3.9,1.4,versicolor
...,...,...,...,...,...
93,5.0,2.3,3.3,1.0,versicolor
98,5.1,2.5,3.0,1.1,versicolor


##### 先頭文字列
---
`pandas.Series.str.startswith`を使用する。  
`pandas.Series.str`や`pandas.Index.str`には他にも文字列操作のためのメソッドが用意されている。

In [70]:
print([att for att in dir(pd.Series.str) if not att.startswith('_')])

['capitalize', 'cat', 'center', 'contains', 'count', 'decode', 'encode', 'endswith', 'extract', 'extractall', 'find', 'findall', 'get', 'get_dummies', 'index', 'isalnum', 'isalpha', 'isdecimal', 'isdigit', 'islower', 'isnumeric', 'isspace', 'istitle', 'isupper', 'join', 'len', 'ljust', 'lower', 'lstrip', 'match', 'normalize', 'pad', 'partition', 'repeat', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'slice', 'slice_replace', 'split', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'wrap', 'zfill']


In [71]:
pd.Series.str.startswith??

In [72]:
mpg.loc[mpg['name'].str.startswith('toyota')]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
14,24.0,4,113.0,95.0,2372,15.0,70,japan,toyota corona mark ii
31,25.0,4,113.0,95.0,2228,14.0,71,japan,toyota corona
...,...,...,...,...,...,...,...,...,...
382,34.0,4,108.0,70.0,2245,16.9,82,japan,toyota corolla
390,32.0,4,144.0,96.0,2665,13.9,82,japan,toyota celica gt


In [73]:
mpg.query('name.str.startswith("toyota")')

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
14,24.0,4,113.0,95.0,2372,15.0,70,japan,toyota corona mark ii
31,25.0,4,113.0,95.0,2228,14.0,71,japan,toyota corona
...,...,...,...,...,...,...,...,...,...
382,34.0,4,108.0,70.0,2245,16.9,82,japan,toyota corolla
390,32.0,4,144.0,96.0,2665,13.9,82,japan,toyota celica gt


##### 部分文字列
---
`pandas.Series.str.contains`を使用する。

In [74]:
pd.Series.str.contains??

In [75]:
mpg.loc[mpg['name'].str.contains('corolla')]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
53,31.0,4,71.0,65.0,1773,19.0,71,japan,toyota corolla 1200
84,27.0,4,97.0,88.0,2100,16.5,72,japan,toyota corolla 1600 (sw)
...,...,...,...,...,...,...,...,...,...
356,32.4,4,108.0,75.0,2350,16.8,81,japan,toyota corolla
382,34.0,4,108.0,70.0,2245,16.9,82,japan,toyota corolla


In [76]:
mpg.query('name.str.contains("corolla")')

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
53,31.0,4,71.0,65.0,1773,19.0,71,japan,toyota corolla 1200
84,27.0,4,97.0,88.0,2100,16.5,72,japan,toyota corolla 1600 (sw)
...,...,...,...,...,...,...,...,...,...
356,32.4,4,108.0,75.0,2350,16.8,81,japan,toyota corolla
382,34.0,4,108.0,70.0,2245,16.9,82,japan,toyota corolla


#### SQL
---
`WHERE`句を使用する。

In [77]:
titanic_db = sqlalchemy.create_engine('sqlite:///data/titanic.sqlite3')
iris_db = sqlalchemy.create_engine('sqlite:///data/iris.sqlite3')
mpg_db = sqlalchemy.create_engine('sqlite:///data/mpg.sqlite3')
iris_duplicated_db = sqlalchemy.create_engine(
    'sqlite:///data/iris_duplicated.sqlite3')

##### 特定の値のデータ

In [78]:
pd.read_sql('SELECT * FROM titanic WHERE Class = "1st"', titanic_db)

,Class,Sex,Age,Survived
0,1st,Male,Adult,No
1,1st,Male,Adult,No
...,...,...,...,...
323,1st,Female,Adult,Yes
324,1st,Female,Adult,Yes


##### 特定の値以外のデータ

In [79]:
pd.read_sql('SELECT Class, Survived FROM titanic WHERE Class != "Crew"',
            titanic_db)

,Class,Survived
0,3rd,No
1,3rd,No
...,...,...
1314,3rd,Yes
1315,3rd,Yes


In [80]:
pd.read_sql('SELECT Class, Survived FROM titanic WHERE Class <> "Crew"',
            titanic_db)

,Class,Survived
0,3rd,No
1,3rd,No
...,...,...
1314,3rd,Yes
1315,3rd,Yes


##### リストに含まれる値を持つデータ

In [81]:
pd.read_sql('SELECT * FROM titanic WHERE Class IN ("1st", "2nd")', titanic_db)

,Class,Sex,Age,Survived
0,1st,Male,Adult,No
1,1st,Male,Adult,No
...,...,...,...,...
608,2nd,Female,Adult,Yes
609,2nd,Female,Adult,Yes


##### リストに含まれる値を除くデータ

In [82]:
pd.read_sql('SELECT * FROM titanic WHERE Class NOT IN ("1st", "2nd")',
            titanic_db)

,Class,Sex,Age,Survived
0,3rd,Male,Child,No
1,3rd,Male,Child,No
...,...,...,...,...
1589,Crew,Female,Adult,Yes
1590,Crew,Female,Adult,Yes


##### 一定の範囲内のデータ
---
`BETWEEN`を使用する。 (両端が含まれる)

In [83]:
pd.read_sql('SELECT * FROM iris WHERE petal_length BETWEEN 2 AND 4', iris_db)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.5,2.3,4.0,1.3,versicolor
1,4.9,2.4,3.3,1.0,versicolor
...,...,...,...,...,...
14,5.0,2.3,3.3,1.0,versicolor
15,5.1,2.5,3.0,1.1,versicolor


両端を含まないようにするには、以下のようにする。

In [84]:
pd.read_sql('SELECT * FROM iris WHERE 2 < petal_length < 4', iris_db)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
...,...,...,...,...,...
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


##### 先頭文字列
---
`LIKE`とワイルドカード`%`を使用する。

In [85]:
pd.read_sql('SELECT * FROM mpg WHERE name LIKE "toyota%"', mpg_db)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,24.0,4,113.0,95.0,2372,15.0,70,japan,toyota corona mark ii
1,25.0,4,113.0,95.0,2228,14.0,71,japan,toyota corona
...,...,...,...,...,...,...,...,...,...
23,34.0,4,108.0,70.0,2245,16.9,82,japan,toyota corolla
24,32.0,4,144.0,96.0,2665,13.9,82,japan,toyota celica gt


##### 部分文字列
---
先頭文字列と同様に`LIKE`とワイルドカード`%`を使用する。

In [86]:
pd.read_sql('SELECT * FROM mpg WHERE name LIKE "%corolla%"', mpg_db)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,31.0,4,71.0,65.0,1773,19.0,71,japan,toyota corolla 1200
1,27.0,4,97.0,88.0,2100,16.5,72,japan,toyota corolla 1600 (sw)
...,...,...,...,...,...,...,...,...,...
8,32.4,4,108.0,75.0,2350,16.8,81,japan,toyota corolla
9,34.0,4,108.0,70.0,2245,16.9,82,japan,toyota corolla


### 重複の除去

In [87]:
iris_duplicated = iris.sample(
    iris.index.size + 10, replace=True,
    random_state=1234).reset_index(drop=True)
iris_duplicated.to_sql(
    'duplicated', iris_duplicated_db, if_exists='replace', index=False)
print('iris_duplicated')
display(iris_duplicated)

iris_duplicated


,sepal_length,sepal_width,petal_length,petal_width,species
0,4.6,3.2,1.4,0.2,setosa
1,4.4,3.0,1.3,0.2,setosa
...,...,...,...,...,...
158,5.0,3.4,1.6,0.4,setosa
159,5.5,2.5,4.0,1.3,versicolor


#### pandas
---
`pandas.DataFrame.drop_duplicates`を使用する。

In [88]:
pd.DataFrame.drop_duplicates??

In [89]:
iris_duplicated.drop_duplicates()

,sepal_length,sepal_width,petal_length,petal_width,species
0,4.6,3.2,1.4,0.2,setosa
1,4.4,3.0,1.3,0.2,setosa
...,...,...,...,...,...
155,6.5,3.0,5.2,2.0,virginica
157,6.2,3.4,5.4,2.3,virginica


#### SQL
---
`DISTINCT`を使用する。

In [90]:
pd.read_sql('SELECT DISTINCT * FROM duplicated', iris_duplicated_db)

,sepal_length,sepal_width,petal_length,petal_width,species
0,4.6,3.2,1.4,0.2,setosa
1,4.4,3.0,1.3,0.2,setosa
...,...,...,...,...,...
91,6.5,3.0,5.2,2.0,virginica
92,6.2,3.4,5.4,2.3,virginica


###### 練習問題

上で保存したアメリカの人口統計データから、以下の条件に合致するデータを pandas と SQL でそれぞれ抽出する。

- Geography カラムの値が New York
- Population Group カラムの値が Total population のものは除外
- Estimate; Total: カラムの値が大きいものから上位 5 件
- 取り出すカラムは Population Group と Estimate; Total:

pandas

In [91]:
census = pd.read_csv('data/ACS_15_SPT_B21001_with_ann.csv', skiprows=1)
census.loc[(census['Geography'] == "New York") &
           (census['Population Group'] != 'Total population'
            ), ['Population Group', 'Estimate; Total:']].sort_values(
                'Estimate; Total:', ascending=False).head(5)

,Population Group,Estimate; Total:
4228,White alone or in combination with one or more...,10426721
4227,White alone,10190219
4365,White alone or in combination with one or more...,9194006
4364,"White alone, not Hispanic or Latino",9056838
4341,Hispanic or Latino (of any race) (200-299),2612604


SQL

In [92]:
engine = sqlalchemy.create_engine('sqlite:///data/census-example.db')
pd.read_sql(
    '''SELECT "Population Group", "Estimate; Total:" FROM census
       WHERE Geography = "New York" AND "Population Group" != "Total population"
       ORDER BY "Estimate; Total:" DESC LIMIT 5''', engine)

,Population Group,Estimate; Total:
0,White alone or in combination with one or more...,10426721
1,White alone,10190219
2,White alone or in combination with one or more...,9194006
3,"White alone, not Hispanic or Latino",9056838
4,Hispanic or Latino (of any race) (200-299),2612604


### ランダムサンプリング
---
- 数十万レコードのデータに対して、ランダムまたは一定間隔にデータを抽出できる
- サンプリングやアンサンブル平均によって適量にデータ量を減らすことができる


#### pandas
---
`pandas.DataFrame.sample`を使用する。

In [93]:
pd.DataFrame.sample??

In [94]:
titanic.sample(10, random_state=1234)

,Class,Sex,Age,Survived
2019,1st,Female,Adult,Yes
860,Crew,Male,Adult,No
...,...,...,...,...
236,2nd,Male,Adult,No
220,2nd,Male,Adult,No


#### SQL
---
乱数を生成する関数を使用して全レコードを並び替え、`LIMIT`で件数を指定して取得する手法もあるが、ランダムサンプリングを行うのは通常レコード数が膨大なときなので、全レコードを並び変えるのは非効率。  
ID などレコードごとに固有の値を持つカラムのみを並び替え、それに基づいてデータを取得する。

In [95]:
titanic_with_id = sqlalchemy.create_engine(
    'sqlite:///data/titanic_with_id.sqlite3')
pd.read_sql('SELECT * FROM titanic LIMIT 5', titanic_with_id)

,id,Class,Sex,Age,Survived
0,1,3rd,Male,Child,No
1,2,3rd,Male,Child,No
2,3,3rd,Male,Child,No
3,4,3rd,Male,Child,No
4,5,3rd,Male,Child,No


In [96]:
ids = pd.read_sql('SELECT id FROM titanic', titanic_with_id).sample(
    10, random_state=1234)['id']
query = f'SELECT * FROM titanic WHERE id IN ({",".join([str(i) for i in ids])})'
print(query)
pd.read_sql(query, titanic_with_id)

SELECT * FROM titanic WHERE id IN (2019,860,1174,368,544,1001,614,802,236,220)


,id,Class,Sex,Age,Survived
0,220,2nd,Male,Adult,No
1,236,2nd,Male,Adult,No
...,...,...,...,...,...
8,1174,Crew,Male,Adult,No
9,2019,1st,Female,Adult,Yes


## データ統合
---
データベースでは以下のように必要な情報が複数のテーブルにまたがっていることがある。

- 共通する情報が別のテーブルにまとめられてる

<table class="text-center border">
    <tr>
        <th class="border-none background-default" colspan="4">customer テーブル (顧客情報)</th>
    </tr>
    <tr>
        <th>id</th>
        <th>氏名</th>
        <th>メールアドレス</th>
        <th>ユーザーランク</th>
    </tr>
    <tr>
        <td>1</td>
        <td>AAA BBB</td>
        <td>aa@bb.com</td>
        <td>S</td>
    </tr>
    <tr>
        <td>2</td>
        <td>XXX YYY</td>
        <td>cc@dd.org</td>
        <td>A</td>
    </tr>
</table>

<table class="text-center border">
    <tr>
        <th class="border-none background-default" colspan="3">item テーブル (商品情報)</th>
    </tr>
    <tr>
        <th>id</th>
        <th>名称</th>
        <th>単価</th>
    </tr>
    <tr>
        <td>1</td>
        <td>M</td>
        <td>1000</td>
    </tr>
    <tr>
        <td>2</td>
        <td>N</td>
        <td>1500</td>
    </tr>
</table>

<table class="text-center border">
    <tr>
        <th class="border-none background-default" colspan="5">sales テーブル (売上情報)</th>
    </tr>
    <tr>
        <th>id</th>
        <th>customer_id</th>
        <th>item_id</th>
        <th>数量</th>
        <th>金額</th>
    </tr>
    <tr>
        <td>1</td>
        <td>1</td>
        <td>2</td>
        <td>2</td>
        <td>3000</td>
    </tr>
    <tr>
        <td>2</td>
        <td>2</td>
        <td>1</td>
        <td>1</td>
        <td>1000</td>
    </tr>
    <tr>
        <td>3</td>
        <td>2</td>
        <td>2</td>
        <td>3</td>
        <td>4500</td>
    </tr>
</table>

- 同種の情報が別のテーブルにまたがっている

<table class="text-center border">
    <tr>
        <th class="border-none background-default" colspan="2">国内売上</th>
    </tr>
    <tr>
        <th>id</th>
        <th>金額</th>
    </tr>
    <tr>
        <td>1</td>
        <td>2500</td>
    </tr>
    <tr>
        <td>2</td>
        <td>1250</td>
    </tr>
    <tr>
        <td>3</td>
        <td>3000</td>
    </tr>
</table>

<table class="text-center border">
    <tr>
        <th class="border-none background-default" colspan="2">海外売上</th>
    </tr>
    <tr>
        <th>id</th>
        <th>金額</th>
    </tr>
    <tr>
        <td>1</td>
        <td>20</td>
    </tr>
    <tr>
        <td>2</td>
        <td>50</td>
    </tr>
</table>

データを分析するにあたっては、これらの中から必要な情報をまとめなければならない。  
テーブル間の関係は ER 図などにまとめられていることがあるので、適宜参照する。

### 結合
---
テーブルを横方向に統合。

#### pandas
---
`pandas.merge`や`pandas.concat`を使用する。  
`pandas.concat`は比較する値が 2 つのテーブルのインデックス同士になる。 (日付をインデックスにしている場合など)

In [97]:
pd.merge??

In [98]:
pd.concat??

##### 内部結合
---
2 つのテーブルのある列同士の値で、両方に同じ値がある行を結合する方法。  

`pandas.merge`

In [99]:
left = pd.read_csv('data/sample.csv', usecols=['A', 'B', 'Key']).iloc[:4]
print('left')
display(left)
right = pd.read_csv(
    'data/sample.csv', usecols=['C', 'D',
                                'Key']).iloc[1:5].reset_index(drop=True)
print('right')
display(right)

left


,A,B,Key
0,A0,B0,K0
1,A1,B1,K1
2,A2,B2,K2
3,A3,B3,K3


right


,C,D,Key
0,C1,D1,K1
1,C2,D2,K2
2,C3,D3,K3
3,C4,D4,K4


In [100]:
pd.merge(left, right, on='Key')

,A,B,Key,C,D
0,A1,B1,K1,C1,D1
1,A2,B2,K2,C2,D2
2,A3,B3,K3,C3,D3


比較対象のカラムには重複する値があっても良い。

In [101]:
left2 = left.append(left.sample(2, random_state=1234)).reset_index(drop=True)
print('left2')
display(left2)
right2 = right.append(right.sample(2,
                                   random_state=2345)).reset_index(drop=True)
print('right2')
display(right2)

left2


,A,B,Key
0,A0,B0,K0
1,A1,B1,K1
...,...,...,...
4,A0,B0,K0
5,A1,B1,K1


right2


,C,D,Key
0,C1,D1,K1
1,C2,D2,K2
...,...,...,...
4,C2,D2,K2
5,C3,D3,K3


In [102]:
pd.merge(left2, right2, on='Key')

,A,B,Key,C,D
0,A1,B1,K1,C1,D1
1,A1,B1,K1,C1,D1
...,...,...,...,...,...
4,A3,B3,K3,C3,D3
5,A3,B3,K3,C3,D3


`pandas.concat`

In [103]:
left = pd.read_csv('data/sample.csv', usecols=['A', 'B', 'C', 'D']).iloc[:4]
print('left')
display(left)
right = pd.read_csv('data/sample.csv', usecols=['B', 'D', 'F']).iloc[[2, 3, 6, 7]]
print('right')
display(right)

left


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


right


,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6
7,B7,D7,F7


In [104]:
pd.concat([left, right], axis=1, join='inner')

,A,B,C,D,B,D,F
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


1 列 (Series オブジェクト) 追加するときにも使用可能。

In [105]:
df = pd.read_csv('data/sample.csv', usecols=['A', 'B', 'C', 'D']).iloc[:4]
print('df')
display(df)
series = pd.Series([f'X{i}' for i in range(df.index.size)])
print('series')
display(series)

df


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


series


0    X0
1    X1
2    X2
3    X3
dtype: object

In [106]:
series.name = 'X'
pd.concat([df, series], axis=1)

,A,B,C,D,X
0,A0,B0,C0,D0,X0
1,A1,B1,C1,D1,X1
2,A2,B2,C2,D2,X2
3,A3,B3,C3,D3,X3


同様のことは以下でも可能。

In [107]:
df['X'] = series
df

,A,B,C,D,X
0,A0,B0,C0,D0,X0
1,A1,B1,C1,D1,X1
2,A2,B2,C2,D2,X2
3,A3,B3,C3,D3,X3


##### 外部結合
---
2 つのテーブルのある列同士の値で、全ての値を用いて結合する方法。  
どちらかにある値を基準に結合する左外部結合・右外部結合もある。 (`pandas.merge`のみ)  
逆のテーブルに値がないところは欠損値になる。

`pandas.merge`

In [108]:
left = pd.read_csv('data/sample.csv', usecols=['A', 'B', 'Key']).iloc[:4]
print('left')
display(left)
right = pd.read_csv(
    'data/sample.csv', usecols=['C', 'D',
                                'Key']).iloc[1:5].reset_index(drop=True)
print('right')
display(right)

left


,A,B,Key
0,A0,B0,K0
1,A1,B1,K1
2,A2,B2,K2
3,A3,B3,K3


right


,C,D,Key
0,C1,D1,K1
1,C2,D2,K2
2,C3,D3,K3
3,C4,D4,K4


In [109]:
pd.merge(left, right, how='outer', on='Key')

,A,B,Key,C,D
0,A0,B0,K0,NaN,NaN
1,A1,B1,K1,C1,D1
2,A2,B2,K2,C2,D2
3,A3,B3,K3,C3,D3
4,NaN,NaN,K4,C4,D4


左外部結合

In [110]:
pd.merge(left, right, how='left', on='Key')

,A,B,Key,C,D
0,A0,B0,K0,NaN,NaN
1,A1,B1,K1,C1,D1
2,A2,B2,K2,C2,D2
3,A3,B3,K3,C3,D3


右外部結合

In [111]:
pd.merge(left, right, how='right', on='Key')

,A,B,Key,C,D
0,A1,B1,K1,C1,D1
1,A2,B2,K2,C2,D2
2,A3,B3,K3,C3,D3
3,NaN,NaN,K4,C4,D4


`pandas.concat`

In [112]:
left = pd.read_csv('data/sample.csv', usecols=['A', 'B', 'C', 'D']).iloc[:4]
print('left')
display(left)
right = pd.read_csv('data/sample.csv', usecols=['B', 'D', 'F']).iloc[[2, 3, 6, 7]]
print('right')
display(right)

left


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


right


,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6
7,B7,D7,F7


In [113]:
pd.concat([left, right], axis=1)

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
...,...,...,...,...,...,...,...
6,NaN,NaN,NaN,NaN,B6,D6,F6
7,NaN,NaN,NaN,NaN,B7,D7,F7


#### SQL

In [114]:
engine = sqlalchemy.create_engine('sqlite:///data/sample.sqlite3')

##### 内部結合
---
`INNER JOIN`と`ON`を使用。

In [115]:
for table in ['left', 'right']:
    print(table)
    display(pd.read_sql(f'SELECT * FROM {table}', engine))

left


,A,B,Key
0,A0,B0,K0
1,A1,B1,K1
2,A2,B2,K2
3,A3,B3,K3


right


,C,D,Key
0,C1,D1,K1
1,C2,D2,K2
2,C3,D3,K3
3,C4,D4,K4


In [116]:
pd.read_sql('SELECT * FROM left INNER JOIN right ON left.Key = right.Key',
            engine)

,A,B,Key,C,D,Key
0,A1,B1,K1,C1,D1,K1
1,A2,B2,K2,C2,D2,K2
2,A3,B3,K3,C3,D3,K3


##### 外部結合
---
`OUTER JOIN`と`ON`を使用。  
SQLite では`FULL OUTER JOIN`と`RIGHT OUTER JOIN`はサポートされていないので、左外部結合のみ。

In [117]:
print('left')
display(pd.read_sql('SELECT * FROM left', engine))
print('right')
display(pd.read_sql('SELECT * FROM right', engine))

left


,A,B,Key
0,A0,B0,K0
1,A1,B1,K1
2,A2,B2,K2
3,A3,B3,K3


right


,C,D,Key
0,C1,D1,K1
1,C2,D2,K2
2,C3,D3,K3
3,C4,D4,K4


In [118]:
pd.read_sql('SELECT * FROM left LEFT OUTER JOIN right ON left.Key = right.Key', engine)

,A,B,Key,C,D,Key
0,A0,B0,K0,None,None,None
1,A1,B1,K1,C1,D1,K1
2,A2,B2,K2,C2,D2,K2
3,A3,B3,K3,C3,D3,K3


### Union処理
---
テーブルを縦方向に統合。

#### pandas
---
`pandas.concat`と`pandas.DataFrame.append`が使用可能。

In [119]:
sample = pd.read_csv('data/sample.csv', usecols=['A', 'B', 'C', 'D'])
first = sample.iloc[:4]
second = sample.iloc[4:8].reset_index(drop=True)
third = sample.iloc[8:].reset_index(drop=True)
for name, df in zip(['first', 'second', 'third'], [first, second, third]):
    print(name)
    display(df)

first


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


second


,A,B,C,D
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7


third


,A,B,C,D
0,A8,B8,C8,D8
1,A9,B9,C9,D9
2,A10,B10,C10,D10
3,A11,B11,C11,D11


`pandas.concat`

In [120]:
pd.concat([first, second, third])

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
...,...,...,...,...
2,A10,B10,C10,D10
3,A11,B11,C11,D11


`pandas.DataFrame.append`  
Python のリストと異なり、**元のオブジェクトを変更せず**新しいオブジェクトを返す。

In [121]:
pd.DataFrame.append??

In [122]:
union = first.append([second, third])

In [123]:
print('first')
first

first


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [124]:
print('union')
union

union


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
...,...,...,...,...
2,A10,B10,C10,D10
3,A11,B11,C11,D11


インデックスを振り直す場合は、`ignore_index`を True にする。

In [125]:
pd.concat([first, second, third], ignore_index=True)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
...,...,...,...,...
10,A10,B10,C10,D10
11,A11,B11,C11,D11


In [126]:
first.append([second, third], ignore_index=True)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
...,...,...,...,...
10,A10,B10,C10,D10
11,A11,B11,C11,D11


#### SQL
---
`UNION ALL`を使用する。 (`UNION`は重複をチェックするがその分コストがかかるので、できる限り避ける)

In [127]:
engine = sqlalchemy.create_engine('sqlite:///data/sample.sqlite3')

In [128]:
for table in ['first', 'second', 'third']:
    print(table)
    display(pd.read_sql(f'SELECT * FROM {table}', engine))

first


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


second


,A,B,C,D
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7


third


,A,B,C,D
0,A8,B8,C8,D8
1,A9,B9,C9,D9
2,A10,B10,C10,D10
3,A11,B11,C11,D11


In [129]:
pd.read_sql(
    '''
    SELECT * FROM first
    UNION ALL
    SELECT * FROM second
    UNION ALL
    SELECT * FROM third
    ''', engine)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
...,...,...,...,...
10,A10,B10,C10,D10
11,A11,B11,C11,D11


###### 練習問題

[アメリカの統計情報](https://factfinder.census.gov/faces/nav/jsf/pages/guided_search.xhtml)から以下の条件で 2 種類のデータを探し、共通してデータがある年度のもののうち、それぞれ最新のものから 2 期分をダウンロードする。

産業統計 (Wholesale Trade: Geographic Area Series: Summary Statistics for the U.S., States, Metro Areas, Counties, and Places)
- Business and Industry:Expenses & Purchased Services: All Expenses
- All States within United States, Puerto Rico, and the Island Areas 
- ALL: All available codes 

人口統計 (Industry by Sex and Median Earnings in the Past 12 Months for the Civilian Employed Population 16 Years and Over)
- People:Basic Count/Estimate: Civilian Population
- All States within United States, Puerto Rico, and the Island Areas 

ダウンロードした CSV ファイルを年度が同じもの同士で州をキーにして内部結合し、異なる年度のものも 1 つのデータフレームにまとめる。それを`data/merge.db`というデータベースに保存する。

In [130]:
engine = sqlalchemy.create_engine('sqlite:///data/merge-example.db')
trade1 = pd.read_csv('data/ECN_2007_US_42A1_with_ann.csv', skiprows=1)
trade1

/root/local/python-3.6.8/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Geographic identifier code,Id2,Geographic area name,2007 NAICS code,Meaning of 2007 NAICS code,Type of operation or tax status code,Meaning of Type of operation or tax status code,Year,Number of establishments,"Sales ($1,000)","Annual payroll ($1,000)","First-quarter payroll ($1,000)",Number of paid employees for pay period including March 12,"Operating expenses ($1,000)","Total inventories, beginning of year ($1,000)","Total inventories, end of year ($1,000)","Sales, receipts, or revenue from administrative records (%)","Sales, receipts, or revenue estimated (%)"
0,0400000US01,1,Alabama,42,Wholesale trade,0,Wholesale Trade,2007,5663,68625052,3600076,873186,81076,6655298,4752268,4713201,3.0,3.2
1,0400000US01,1,Alabama,42,Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",2007,4824,52252752,3032207,736016,70469,5580455,4492504,4440758,3.6,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35980,0400000US56,56,Wyoming,42512047,Petroleum and petroleum products agents and br...,0,Wholesale Trade,2007,1,D,D,D,a,D,D,D,D,D
35981,0400000US56,56,Wyoming,42512048,"Beer, wine, and distilled alcoholic beverage a...",0,Wholesale Trade,2007,1,D,D,D,a,D,D,D,D,D


In [131]:
trade1.columns = [c.replace('2007 ', '') for c in trade1.columns]

In [132]:
trade2 = pd.read_csv('data/ECN_2012_US_42A1_with_ann.csv', skiprows=1)
trade2

,Geographic identifier code,Id2,Geographic area name,2012 NAICS code,Meaning of 2012 NAICS code,Type of operation or tax status code,Meaning of Type of operation or tax status code,Year,Number of establishments,"Sales ($1,000)","Annual payroll ($1,000)","First-quarter payroll ($1,000)",Number of paid employees for pay period including March 12,"Operating expenses ($1,000)","Total inventories, beginning of year ($1,000)","Total inventories, end of year ($1,000)",Percent of sales from administrative records (%),Percent of sales estimated (%)
0,0400000US01,1,Alabama,42,Wholesale trade,0,Wholesale Trade,2012,5408,75844516,3669258,901695,73312,7041507,4643703,4883343,4.0,3.4
1,0400000US01,1,Alabama,42,Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",2012,4600,57746565,2894745,706805,60332,5646163,4375881,4607539,5.1,4.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35777,0400000US56,56,Wyoming,42512048,"Beer, wine, and distilled alcoholic beverage a...",0,Wholesale Trade,2012,1,D,D,D,a,D,D,D,D,D
35778,0400000US56,56,Wyoming,42512049,Miscellaneous nondurable goods agents and brokers,0,Wholesale Trade,2012,1,D,D,D,a,D,D,D,D,D


In [133]:
trade2.columns = [c.replace('2012 ', '') for c in trade2.columns]

In [134]:
population1 = pd.read_csv('data/ACS_07_1YR_S2403_with_ann.csv', skiprows=1)
population1

,Id,Id2,Geography,Total; Estimate; Civilian employed population 16 years and over,Total; Margin of Error; Civilian employed population 16 years and over,Male; Estimate; Civilian employed population 16 years and over,Male; Margin of Error; Civilian employed population 16 years and over,Female; Estimate; Civilian employed population 16 years and over,Female; Margin of Error; Civilian employed population 16 years and over,Median earnings (dollars); Estimate; Civilian employed population 16 years and over,...,Male; Estimate; Civilian employed population 16 years and over - PERCENT IMPUTED - Industry,Male; Margin of Error; Civilian employed population 16 years and over - PERCENT IMPUTED - Industry,Female; Estimate; Civilian employed population 16 years and over - PERCENT IMPUTED - Industry,Female; Margin of Error; Civilian employed population 16 years and over - PERCENT IMPUTED - Industry,Median earnings (dollars); Estimate; Civilian employed population 16 years and over - PERCENT IMPUTED - Industry,Median earnings (dollars); Margin of Error; Civilian employed population 16 years and over - PERCENT IMPUTED - Industry,Median earnings (dollars) for male; Estimate; Civilian employed population 16 years and over - PERCENT IMPUTED - Industry,Median earnings (dollars) for male; Margin of Error; Civilian employed population 16 years and over - PERCENT IMPUTED - Industry,Median earnings (dollars) for female; Estimate; Civilian employed population 16 years and over - PERCENT IMPUTED - Industry,Median earnings (dollars) for female; Margin of Error; Civilian employed population 16 years and over - PERCENT IMPUTED - Industry
0,0400000US01,1,Alabama,2027526,15894,53.5,0.3,46.5,0.3,28221,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
1,0400000US02,2,Alaska,329656,4555,54.3,0.7,45.7,0.7,35502,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,0400000US56,56,Wyoming,278964,4846,54.2,0.8,45.8,0.8,29884,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
51,0400000US72,72,Puerto Rico,1184129,14726,53.7,0.6,46.3,0.6,15561,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)


In [135]:
population2 = pd.read_csv('data/ACS_12_1YR_S2403_with_ann.csv', skiprows=1)
population2

,Id,Id2,Geography,Total; Estimate; Civilian employed population 16 years and over,Total; Margin of Error; Civilian employed population 16 years and over,Male; Estimate; Civilian employed population 16 years and over,Male; Margin of Error; Civilian employed population 16 years and over,Female; Estimate; Civilian employed population 16 years and over,Female; Margin of Error; Civilian employed population 16 years and over,Median earnings (dollars); Estimate; Civilian employed population 16 years and over,...,Male; Estimate; PERCENT IMPUTED - Industry,Male; Margin of Error; PERCENT IMPUTED - Industry,Female; Estimate; PERCENT IMPUTED - Industry,Female; Margin of Error; PERCENT IMPUTED - Industry,Median earnings (dollars); Estimate; PERCENT IMPUTED - Industry,Median earnings (dollars); Margin of Error; PERCENT IMPUTED - Industry,Median earnings (dollars) for male; Estimate; PERCENT IMPUTED - Industry,Median earnings (dollars) for male; Margin of Error; PERCENT IMPUTED - Industry,Median earnings (dollars) for female; Estimate; PERCENT IMPUTED - Industry,Median earnings (dollars) for female; Margin of Error; PERCENT IMPUTED - Industry
0,0400000US01,1,Alabama,2001628,16432,52.7,0.3,47.3,0.3,30221,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
1,0400000US02,2,Alaska,351496,5687,53.3,0.7,46.7,0.7,39070,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,0400000US56,56,Wyoming,288852,4610,54.0,0.8,46.0,0.8,32428,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
51,0400000US72,72,Puerto Rico,1094779,12433,53.0,0.5,47.0,0.5,17344,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)


In [136]:
merge1 = pd.merge(trade1, population1, on='Id2')
merge2 = pd.merge(trade2, population2, on='Id2')
merge = pd.concat([merge1, merge2], join='inner')
merge

,Geographic identifier code,Id2,Geographic area name,NAICS code,Meaning of NAICS code,Type of operation or tax status code,Meaning of Type of operation or tax status code,Year,Number of establishments,"Sales ($1,000)",...,Male; Estimate; Civilian employed population 16 years and over,Male; Margin of Error; Civilian employed population 16 years and over,Female; Estimate; Civilian employed population 16 years and over,Female; Margin of Error; Civilian employed population 16 years and over,Median earnings (dollars); Estimate; Civilian employed population 16 years and over,Median earnings (dollars); Margin of Error; Civilian employed population 16 years and over,Median earnings (dollars) for male; Estimate; Civilian employed population 16 years and over,Median earnings (dollars) for male; Margin of Error; Civilian employed population 16 years and over,Median earnings (dollars) for female; Estimate; Civilian employed population 16 years and over,Median earnings (dollars) for female; Margin of Error; Civilian employed population 16 years and over
0,0400000US01,1,Alabama,42,Wholesale trade,0,Wholesale Trade,2007,5663,68625052,...,53.5,0.3,46.5,0.3,28221,465,34183,624,22151,289
1,0400000US01,1,Alabama,42,Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",2007,4824,52252752,...,53.5,0.3,46.5,0.3,28221,465,34183,624,22151,289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35777,0400000US56,56,Wyoming,42512048,"Beer, wine, and distilled alcoholic beverage a...",0,Wholesale Trade,2012,1,D,...,54.0,0.8,46.0,0.8,32428,1687,45145,3108,23668,1473
35778,0400000US56,56,Wyoming,42512049,Miscellaneous nondurable goods agents and brokers,0,Wholesale Trade,2012,1,D,...,54.0,0.8,46.0,0.8,32428,1687,45145,3108,23668,1473


In [137]:
merge.to_sql('merge', engine, if_exists='replace')

## インターネットからのデータ収集
---
手元にあるデータだけでは不十分な場合に、インターネットから情報を集めることはよくある。その際に Python で使用する主なパッケージを以下に示す。

### [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
---
HTML から情報を取り出すためのライブラリ。  
CSS セレクタを使って要素を選択できる。

### [Scrapy](https://scrapy.org/)
---
ウェブサイトを巡回してデータを取得するスクレイピングのためのフレームワーク。

### [Selenium](https://www.seleniumhq.org/docs/)
---
Web ブラウザを操作するためのライブラリ。  
主に JavaScript を使用した動的なサイトから情報を取得する場合に使用。

### [Splash](https://splash.readthedocs.io/en/stable/)
---
Selenium と同じく Web ブラウザを操作するためのものだが、 HTTP API を備えた Docker のサービスとして提供。  
画像の読み込みを無効化したりできる。